# Libraries

In [17]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from scipy import stats
import warnings

from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer

warnings.filterwarnings('ignore')

# Loading Dataset

In [2]:
data = pd.read_excel("dataset.xlsx")

In [3]:
data.shape

(29, 2)

# EDA

In [4]:
data.head()

,Sentence,Outcome
0,What is the difference between depression and ...,C
1,What are other psychiatric conditions that can...,C
2,Why is depression more prevalent in women than...,C
3,What happens during menopause with regards to ...,C
4,What efforts are underway to improve treatment...,C


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  29 non-null     object
 1   Outcome   29 non-null     object
dtypes: object(2)
memory usage: 592.0+ bytes


In [6]:
for col in data.columns:
    print(data[col].value_counts())

What is the difference between depression and sadness?                                                                1
What are some of the treatments for depression?                                                                       1
What is Postpartum Depression (PPD)?                                                                                  1
What are some of the signs and symptoms of depression?                                                                1
What is depression?                                                                                                   1
What is postpartum depression?                                                                                        1
Can depression affect children?                                                                                       1
Who is at risk of depression?                                                                                         1
What are the causes of depression?      

In [7]:
data['Outcome'].value_counts()

P    8
C    7
I    7
O    7
Name: Outcome, dtype: int64

In [8]:
data.dtypes

Sentence    object
Outcome     object
dtype: object

In [9]:
data['Outcome'].value_counts()/np.float(len(data))

P    0.275862
C    0.241379
I    0.241379
O    0.241379
Name: Outcome, dtype: float64

In [10]:
print(round(data.describe(),4))

                                                 Sentence Outcome
count                                                  29      29
unique                                                 29       4
top     What is the difference between depression and ...       P
freq                                                    1       8


# Feature Vector and Target Variable

In [11]:
x=data.drop(["Outcome"],axis=1)
y=data["Outcome"]

# Splitting Training and Testing Dataset

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state = 0)

In [13]:
x_train.shape, x_test.shape

((21, 1), (8, 1))

In [14]:
x_train.head()

,Sentence
5,Why have researchers so far failed to identify...
14,Can depression affect physical health?
8,How is depression diagnosed and treated?
16,What about depression later in life?
22,Who is at risk of depression?


In [16]:
x_test.head()

,Sentence
2,Why is depression more prevalent in women than...
20,Could my depression be causing me physical pain?
23,Can depression affect children?
13,Do I need medicine to treat my depression?
11,"What ideas are on the horizon for better, more..."


In [18]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000) 

In [26]:
x_train = vectorizer.fit_transform(x_train[0])
x_train= x_train.toarray()
x_train.shape

AttributeError: lower not found